In [1]:
#this is the encoder sim_c
#set up a state space model sort of thing
from riglib.bmi.state_space_models import StateSpaceEndptVel2D
ssm = StateSpaceEndptVel2D()

# specify a control matrix
N_NEURONS = 20
N_STATES = 7  # 3 positions and 3 velocities and an offset


# build the observation matrix
sim_C = np.zeros((N_NEURONS, N_STATES))
# control x positive directions
sim_C[2, :] = np.array([0, 0, 0, 1, 0, 0, 0])
sim_C[3, :] = np.array([0, 0, 0, -1, 0, 0, 0])
# control z positive directions
sim_C[5, :] = np.array([0, 0, 0, 0, 0, 1, 0])
sim_C[6, :] = np.array([0, 0, 0, 0, 0, -1, 0])

In [2]:
#  for initialization, we just train the decoder, right.
#initialize the goal estimator
#fb_ctrl = LQRController(A, B, Q, R)

#initialize the encoder
import riglib.bmi.sim_neurons as sim_neurons
np.random.seed(seed= 6)
encoder = sim_neurons.GenericCosEnc(sim_C, ssm,
                                           call_ds_rate=1) #for sim, we want this to return us every call, 

intended_state = np.asarray([0, 0, 0, 10, 0, 10, 1]).T #transpose into a col vector
neural_obs = encoder(intended_state)

#initialize the decoder
from features import simulation_features
decoder =  simulation_features.SimKFDecoderSup()
decoder.load_decoder(supplied_encoder=encoder, 
                    supplied_SSM=ssm)

print('decoder decoded state')
print(decoder.decoder(neural_obs))

/home/sijia-aw/BMi3D_my/lab_bmi3d/riglib/blackrock/cerelink.py:13: UserWarning: Unable to import cerebus library. Check if is installed if using the Blackrock NeuroPort system
  warnings.warn("Unable to import cerebus library. Check if is installed if using the Blackrock NeuroPort system")


Creating simulation decoder..
SimKFDecoderSup:loaded suppled_encoder from function input
<riglib.bmi.sim_neurons.GenericCosEnc object at 0x7f48d81cb040> <class 'riglib.bmi.sim_neurons.GenericCosEnc'>
SimKFDecoderSup:loaded suppled_ssm from function input
<riglib.bmi.sim_neurons.GenericCosEnc object at 0x7f48d81cb040> <class 'riglib.bmi.sim_neurons.GenericCosEnc'>
SimKFDecoderSup: units:  20
 SimKFDecoderSup: zscore decoder ? :  False
{}
end of kwargs
('zscore value: ', False, <class 'bool'>)
no init_zscore
decoder decoded state
[[0.        ]
 [0.        ]
 [0.        ]
 [2.62700103]
 [0.        ]
 [2.77488901]
 [1.        ]]


In [3]:
from riglib.bmi.feedback_controllers import LQRController
Q = np.mat(np.diag([1., 1, 1, 0, 0, 0, 0]))
R = 10000*np.mat(np.diag([1., 1., 1.]))

current_state = decoder.decoder.get_state()
print('current_state')
print(current_state)


print('intended state:')
print(intended_state)

A,B,W = ssm.get_ssm_matrices()
fb_ctrl = LQRController(A, B, Q, R)
ctrl = fb_ctrl.calc_next_state(current_state, intended_state)
print('control input into the state')
print(ctrl)

print('encoder generate control')
print(encoder(ctrl))

current_state
[0.         0.         0.         2.62700103 0.         2.77488901
 1.        ]
intended state:
[ 0  0  0 10  0 10  1]
control input into the state
[[0.2627001 ]
 [0.        ]
 [0.2774889 ]
 [4.26997836]
 [0.        ]
 [4.34479532]
 [1.        ]]
encoder generate control
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
from riglib.bmi.clda import KFSmoothbatch

sb_updater = KFSmoothbatch(batch_time = 20,
                          half_life = 10)